In [1]:
import numpy as np
import pandas as pd
from itertools import product
import os

### Peak information from ATAC-seq data

In [27]:
peak_region = pd.read_csv('../results/peak_regions.bed', sep='\t', header=None)
peak_region['center'] = round((peak_region[2] + peak_region[3])/2).astype('int')
peak_region['chr'] = peak_region[0].str.split(r':|-',expand=True)[0]
peak_region = peak_region[['chr', 0, 'center']]
peak_region.columns = ['chr', 'peak_name', 'center']
peak_region['index'] =  [x for x in range(peak_region.shape[0])]
peak_region.head(5)

,chr,peak_name,center,index
0,GL000194.1,GL000194.1:114519-115365,114942,0
1,GL000194.1,GL000194.1:55758-56597,56178,1
2,GL000194.1,GL000194.1:58217-58957,58587,2
3,GL000194.1,GL000194.1:59535-60431,59983,3
4,GL000195.1,GL000195.1:119766-120427,120096,4


In [34]:
peak_region.to_csv('../results/peak_regions_center.txt', index=False, sep = '\t')

### Gene information from RNA-seq data

In [4]:
DATA_DIR = "/Users/xzeng/Desktop/kaggle/open-problems-multimodal"
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")

start, stop = 0, 10

multi_train_y = pd.read_hdf(FP_MULTIOME_TRAIN_TARGETS, start=start, stop=stop)
# get gene name from multiome data
y_columns = multi_train_y.columns.to_list()
print(len(y_columns))

23418


### Gene annotation informtion from GTF file

In [5]:
# read the gtf annotation file
gtf_anno = pd.read_csv('../reference/gencode.v32.primary_assembly.annotation_gene.bed', sep='\t', header=None)
gtf_anno = gtf_anno.loc[:,[0,1,2,3,5]]
gtf_anno.columns = ['chr', 'start', 'end', 'gene_name', 'strand']
gtf_anno['gene_name'] = gtf_anno['gene_name'].str.split('.',expand = True)[[0]]
gtf_anno.head(5)

,chr,start,end,gene_name,strand
0,GL000009.2,56139,58376,ENSG00000278704,-
1,GL000194.1,53589,115018,ENSG00000277400,-
2,GL000194.1,53593,115055,ENSG00000274847,-
3,GL000195.1,37433,37534,ENSG00000277428,-
4,GL000195.1,42938,49164,ENSG00000276256,-


In [6]:
# remove duplicates
gtf_anno_filter = gtf_anno[gtf_anno['gene_name'].isin(y_columns)]
gtf_anno_filter = gtf_anno_filter[~gtf_anno_filter.duplicated(subset='gene_name')]
gtf_anno_filter.shape

(23418, 5)

In [28]:
# get the transcription start site (tss) from gtf file
gtf_anno_filter_copy = gtf_anno_filter.copy()
gtf_anno_filter_copy['tss'] = ''
# if a gene in the sense (+) strand, the tss is in the start, else if in the antisense (-) strand, the tss is in the end. 
gtf_anno_filter_copy.loc[gtf_anno_filter_copy['strand'] == '+','tss'] = gtf_anno_filter[gtf_anno_filter['strand'] == '+']['start']
gtf_anno_filter_copy.loc[gtf_anno_filter_copy['strand'] == '-','tss'] = gtf_anno_filter[gtf_anno_filter['strand'] == '-']['end']
gene_tss = gtf_anno_filter_copy[['chr', 'gene_name', 'tss']].reset_index(drop=True)
gene_tss.tss = gene_tss.tss.astype('int')
gene_tss['index'] = [x for x in range(gene_tss.shape[0])]
gene_tss.head(5)

,chr,gene_name,tss,index
0,GL000009.2,ENSG00000278704,58376,0
1,GL000194.1,ENSG00000277400,115018,1
2,GL000194.1,ENSG00000274847,115055,2
3,GL000195.1,ENSG00000276256,49164,3
4,GL000195.1,ENSG00000278198,173871,4


In [33]:
gene_tss.to_csv('../results/gene_position.txt', index=False, sep = '\t')

### Calculate the distance between genes and peaks 

In [35]:
def cal_distance(gene_info, peak_info, threshold = 80000):
    
    chr_list = list(set(gene_info['chr'].tolist()))
    gene_peak_dist_lists=[]
    for x in chr_list:
        
        gene_temp = gene_info[gene_info.chr == x][['index', 'tss']]
        peak_temp = peak_info[peak_info.chr == x][['index', 'center']]
        
        gene_name_list = gene_temp['index'].tolist()
        peak_name_list = peak_temp['index'].tolist()
        
        print(x)
        loop_val = [gene_name_list, peak_name_list]
        t = 0
        for i in product(*loop_val):
            
            i = list(i)
            distance = abs(gene_temp.loc[i[0],'tss'] - peak_temp.loc[i[1],'center'])
            
            if distance < threshold:
                i.append(distance)
                gene_peak_dist_lists.append(i)
            else:
                pass
            t += 1 
            if t % 10000000 == 0:
                print(t)

    return gene_peak_dist_lists

In [36]:
gene_peak_dist_df = cal_distance(gene_tss, peak_region)

KI270721.1
chr9
chr21
chr10
chr3
10000000
20000000
GL000219.1
chr18
chr16
chr20
chr7
10000000
chrY
chr6
10000000
chr8
GL000218.1
chr11
10000000
chr12
10000000
GL000009.2
KI270711.1
chr1
10000000
20000000
30000000
40000000
GL000194.1
GL000205.2
chrX
chr14
chr15
KI270734.1
chr13
chr4
10000000
GL000195.1
chr2
10000000
20000000
30000000
chr5
10000000
chr19
10000000
chrM
chr17
10000000
chr22


In [58]:
gene_peak_dist_df[0:5]

[[9, 50, 50],
 [9, 51, 6234],
 [21808, 213385, 30381],
 [21808, 213535, 32183],
 [21808, 213749, 62541]]

In [53]:
gene_index = []
peak_index = []
distance = []
for gene_peak_distances in gene_peak_dist_df:
    gene_index.append(gene_peak_distances[0])
    peak_index.append(gene_peak_distances[1])
    distance.append(gene_peak_distances[2])

In [54]:
gene_peak_dist_df_s = pd.DataFrame({'gene_index': gene_index,
                                   'peak_index': peak_index,
                                   'distance' : distance})

In [59]:
gene_peak_dist_df_s.to_csv('../results/gene_peak_dist_within80K.txt', sep='\t', index=False, header=True)